In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import datetime as dt
from sklearn.linear_model import BayesianRidge
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from pykrige.ok import OrdinaryKriging
import time

In [4]:
df = pd.read_csv('data/intermediate/td_data.csv')
stations = gpd.read_file('data/intermediate/stations.shp')

Now breaking the stations down and looking at missing CO values as well as length of time reported for (26280 reports is 3 years). Some stations report 100 % missing values for their whole time periods which will be neglected from the set. Those remaining are present for much more of the year as well as having almost complete NOx and NO2 reading which could be used to predict some of the missing CO values. However two of the remaining stations are missing O3, PM10 and SO2 for the whole time period, and in addition they have only reported for about a third of the time period, therefore these will also be dropped.

In [8]:
missing_percent = df[df['station'] != 0].isnull().sum().sort_values() / len(df[df['station'] != 0])
predictors = list(missing_percent[(missing_percent <= missing_percent['CO']) & (missing_percent > 0)].index)

def get_null_perc(x, dfin, keep_length=False, grpby=['station']):
    xna = '_'.join([x, 'na'])
    df = dfin.copy()
    df[xna] = df[x].isnull()
    
    sm = df[['station', xna, 'date']].groupby(grpby).agg(
        length=(xna, 'size'),
        na_sum=(xna, 'sum'), 
        mindate=('date', 'min'), 
        maxdate=('date', 'max')
    )
    
    sm['_'.join([xna.lower(), 'perc'])] = sm['na_sum'] / sm['length'] * 100
    sm = sm.filter(regex='perc|date|length').reset_index()
    return(sm)

dfin = df

sm = get_null_perc(predictors[0], dfin)
for x in predictors[1:]:
    sm = sm.merge(get_null_perc(x, dfin), on=['station', 'length', 'mindate', 'maxdate'], how='inner')

# print(sm)

# Filtering out stations with with large number of missing values for CO and missing parts of the year.
chosen_station_id = sm.loc[(sm['co_na_perc'] < 30), 'station'].values.tolist() + [0]

Merging on the chosen station information.

In [9]:
chosen_stations = stations[stations['station'].isin(chosen_station_id)]

keep = ['station', 'date', 'lon', 'lat', 'elevation', 'sin_time', 'cos_time', 'sin_month', 'cos_month']
df2 = df.merge(chosen_stations[['station', 'lon', 'lat', 'elevation']], on='station', how='inner')[keep + predictors]

In [ ]:
dft = df2.copy()

X = dft[dft['station'] != 0].to_numpy()

start = time.time()

imputer = IterativeImputer(max_iter=100, 
                           random_state=0, 
                           tol=1e-4, 
                           skip_complete=True,
                           estimator=BayesianRidge())

X_p = np.concatenate((X[:, :2], imputer.fit_transform(X[:, 2:])), axis=1)

print(time.time() - start)

df3 = pd.DataFrame().from_records(X_p)
df3.columns = df2.columns.tolist()

df3['NA'] = np.where(df2.isnull(), 1, 0)

df3

After comparing 3d kriging (using lon, lat, elevation) with the 2d kriging (lon, lat) the difference in estimate is very small (~%) and therefore due to the execution time being doubled I have opted to use 2d in the interest of time (though the 3d has also been included). Some of the values for PM10 (which have not been imputed) are 5 mg/m3 for all stations. This creates an error for the krigging and therefore the value for the ADL HQ has been given the same value (this is most likely due to the concentration being below a minimum read).

In [ ]:
df7 = df6.copy()
df7 = df7.append(dft[dft['station'] == 0], ignore_index=True)

adl_loc = stations[stations['name'] == 'ADL HQ'][['lon', 'lat', 'elevation']]
adl_lon = adl_loc['lon']
adl_lat = adl_loc['lat']
adl_elev = adl_loc['elevation']

start = time.time()

for p in predictors:
    for d in df6.date.unique():
        try:
            OK = OrdinaryKriging(
                df6.loc[df6.date == d, 'lon'].values,
                df6.loc[df6.date == d, 'lat'].values,
                df6.loc[df6.date == d, p].values,
                coordinates_type='geographic',
                variogram_model='linear'
            )
            z, ss = OK.execute('points', adl_lon, adl_lat)
        except:
            print(p)
            print(d)
            l = df6.loc[df6.date == d, p].values.tolist()
            if np.mean(l) - l[0] == 0:
                z = np.mean(l)
            else:
                z = np.NaN
            
        df7.loc[(df7.date == d) & (df7.station == 0), p] = z
    print(p)
    print(time.time() - start)